<a href="https://colab.research.google.com/github/EkaterinaVZ/hackathon_2023/blob/main/part2__training_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Хакатон_2023_20.01.2023

* загружаем ранее подготовленный датасет с собранной информацией из json + сущности из docx и датасет с текстами из документов<br>
* размечаем данные для обучения spacy новым NER<br>
* проверяем новые документы на наличие в них ошибок

Имеющаяся информация:
* id,
* 'ORG' - название организации,
* 'signatory' - подписант
* 'date_sig' - дата подписания,
* 'position_of_signatory' - должность подписанта,
* 'title' - название документа,
* 'Task1.task_text' - текст задачи,
* 'Task1.task_responsibles_people' - ответствнный за задачу,
* 'Task1.task_responsibles_groups' - ответственный отдел за задачу,
* 'Task1.task_deadline' - срок выполнения задачи, (и так по каждой задаче),
* 'Global_supervisor' - главный ответственный,
* 'Global_deadline' - общий срок выполнения

### Ипортируем необходимые библиотеки

In [1]:
! pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=696d73994ea517038c95d761f0b9c77473fbb62bc4cb44ec296d87600e818548
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [2]:
import os
import zipfile

import docx
from termcolor import colored

import re

# для работы с графами
import networkx as nx

import json
import pandas as pd

In [3]:
! pip install pullenti-wrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.6 MB/s eta 0:00:00


In [4]:
from pullenti_wrapper.processor import (
    Processor,
    ORGANIZATION,
    DATE,
    GEO,
    PERSON,
    MONEY,
    PHONE, 
    ADDRESS
)

In [5]:
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 111.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6668cae707d0c3ab41c78254d7ecd0f863d4c82047f35ec4bce3235e122356df
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [35]:
import spacy
nlp = spacy.load('ru_core_news_lg')

# Загружаем подготовленные ранее данные для разметки

In [7]:
df1 = pd.read_csv("/content/drive/MyDrive/Хакатон_2023/markup_18.01.2023.csv")

In [8]:
df1.head(3)

,id,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,...,responsibles_people_Task4.,responsibles_people_Task5.,responsibles_people_Task6.,responsibles_people_Task7.,responsibles_people_Task8.,responsibles_people_Task9.,responsibles_people_respon,Nasmes_global_supervisor,Names_global_supervisor,Posts_global_supervisors
0,0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"['19 апр 15', 1429383600.0]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Кравцову И.Е.'],['специалиста первой категории']
1,1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"['24 октября 12', 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Кима Юрия Ивановича', 'Сергеева Андрея', 'Ко...","['специалиста третьей категории', 'системного ..."
2,2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"['24.12.20', 1608750000.0]",Провести рефакторинг программного обеспечения ...,...,технического писателя,специалистом третьей категории,специалиста третьей категории,NaN,NaN,NaN,NaN,NaN,"['Абаимовым Симоном', 'Ответственным', 'Абаимо...","['тимлидами проектной команды', 'тимлид проект..."


In [10]:
df2 = pd.read_csv("/content/drive/MyDrive/Хакатон_2023/text.csv")
df2.set_index('id', inplace=True)
df2.head(3)

,text
id,
0,"ООО ""Мегасофт""\n\n\nПриказ\nДля организации ме..."
1,"ООО ""Мегасофт""\n\n\nПриказ\nДля организации ме..."
2,"ООО ""Мегасофт""\n\n\nПриказ\nВ связи с окончани..."


In [13]:
nlp = spacy.load('/content/drive/MyDrive/Хакатон_2023/spacy/ner_sig4')

In [14]:
def get_post(x):
  doc = nlp(x)
  for ent in doc.ents:
    print(ent)
    return ent.text

In [16]:
df1["posts"] = df2["text"][:2].apply(lambda x: get_post(x))

и.о. генерального директора Сердитый А.И.

и.о. генерального директора Сердитый А.И.



In [31]:
def clear_text(x):
    txt = ""
    s = re.sub("\s+", " ", x)
    doc = nlp(s)
    for entity in doc:
        res = entity.lemma_
        txt += (res + " ")
    return txt.rstrip(" ")

In [12]:
df2["clear_text"] = df2["text"].apply(lambda x: clear_text(x))

In [ ]:
df2.to_csv(".csv")

In [17]:
# def clear_txt(x):
#   txt = ""
#   doc = nlp(x)
#   for entity in doc:
#       res = entity.lemma_
#       txt += (" " + res)
#   return txt


In [36]:
groups = [''.join(clear_text(x)) for x in df1['Task1.task_responsibles_groups']] # список строк из df1 с отделами для сущности GROUPS

In [38]:
for i in df1.columns:
  if "responsibles_people" in i:
    posts = [''.join(clear_text(x)) for x in df1[i] if x and isinstance(x, str)] # список для сущности POST

In [ ]:
titles = [''.join(clear_text(x)) for x in df1['Title']] # список для сущности Title

In [ ]:
sig = [''.join(clear_text(x)) for x in df1['Signatory']] # список для сущности Signatory

In [ ]:
for i in df1.columns:
  if "task_responsibles" in i:
    tasks = [''.join(clear_text(x)) for x in df1[i] if x] # список строк для сущности TASK

In [39]:
tasks = [''.join(clear_text(x)) for x in df1["Task1.task_text"][:100] if x]

### Тренировочные данные для spacy

In [41]:
def get_entities(x, list_name, ner):
    
    for i in range(len(list_name)):
      for m in re.finditer(list_name[i].strip(), x):         
              return (m.start(), m.end(), ner)
      

In [42]:
train_groups = []
train_posts = []
# train_titles = []
# train_signatories = []
train_tasks = []

In [ ]:
tasks = [''.join(clear_text(x)) for x in df1["Task1.task_text"] if x and isinstance(x, type(str))] # список строк для сущности TASK

In [43]:
df2["clear_text"].apply(lambda x: train_groups.append((x, {"entities": [get_entities(x, groups, "GROUP")]}))) # создаем тренировочные данные для сущности GROUP
df2["clear_text"].apply(lambda x: train_posts.append((x, {"entities": [get_entities(x, posts, "POST")]}))) # создаем тренировочные данные для сущности POST
# df2["clear_text"].apply(lambda x: train_titles.append((x, {"entities": [get_entities(x, titles, "TITLE")]}))) # создаем тренировочные данные для сущности TITLE
# df2["clear_text"].apply(lambda x: train_signatories.append((x, {"entities": [get_entities(x, sig, "SIG")]}))) # создаем тренировочные данные для сущности SIG
df2["clear_text"][:250].apply(lambda x: train_tasks.append((x, {"entities": [get_entities(x, tasks, "TASK")]}))) # создаем тренировочные данные для сущности TASK

id
0      None
1      None
2      None
3      None
4      None
       ... 
245    None
246    None
247    None
248    None
249    None
Name: clear_text, Length: 250, dtype: object

In [44]:
train_groups[:2]

[(' ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : a. закупить книга . срок завершение : 19 апр 15 . ответственность за исполнение распоряжение возложить на специалист первый категория административный - хозяйственный отдел кравцов и.е. и.о. генеральный директор сердитый а.и. 11 апрель 2013 год',
  {'entities': [(219, 257, 'GROUP')]}),
 (' ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : 1 . провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 . ответственность за исполнение распоряжение возложить на специалист третий категория служба ит ким юрий иванович 2 . выполнить ремонт кабель сеть передача данных между этаж здание . дата окончание : 12 авг 13 . контроль над выполнение приказ возложить на системный администратор сергеев андрей 3 . создать и настроить виртуальный машина в соответствие с технический задание . установить срок выполнение задача до 10 сен 19 . контроль

In [45]:
train_posts[:2]

[(' ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : a. закупить книга . срок завершение : 19 апр 15 . ответственность за исполнение распоряжение возложить на специалист первый категория административный - хозяйственный отдел кравцов и.е. и.о. генеральный директор сердитый а.и. 11 апрель 2013 год',
  {'entities': [(191, 218, 'POST')]}),
 (' ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : 1 . провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 . ответственность за исполнение распоряжение возложить на специалист третий категория служба ит ким юрий иванович 2 . выполнить ремонт кабель сеть передача данных между этаж здание . дата окончание : 12 авг 13 . контроль над выполнение приказ возложить на системный администратор сергеев андрей 3 . создать и настроить виртуальный машина в соответствие с технический задание . установить срок выполнение задача до 10 сен 19 . контроль 

In [46]:
train_groups = [x for x in train_groups if x[1]['entities'][0] != None]
train_posts = [x for x in train_posts if x[1]['entities'][0] != None]
# train_titles = [x for x in train_titles if x[1]['entities'][0] != None]
# train_signatory = [x for x in train_signatories if x[1]['entities'][0] != None]
train_tasks = [x for x in train_tasks if x[1]['entities'][0] != None]

In [47]:
for i in train_tasks[:5]:
  # print(i)
    print(i[0][i[1]['entities'][0][0]:i[1]['entities'][0][1]])
  

закупить книга . срок завершение : 19 апр 15 .
провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 .
провести рефакторинг программный обеспечение для повышение производительность . срок выполнение поручение : 24.12.20 .
оплатить командировочный расход . определить срок исполнение задача вплоть до 26 июль 2019 г ..
согласовать смета на выполнение работа компания подрядчик . привести вышеизложенный поручение в исполнение к 12 янв 2019 г ..


In [53]:
for i in train_groups:
  # print(i)
    print(i[0][i[1]['entities'][0][0]:i[1]['entities'][0][1]])

Выходные данные были обрезаны до нескольких последних строк (5000).
бухгалтерия
административный - хозяйственный отдел
бухгалтерия
административный - хозяйственный отдел
отдел проектирование
административный - хозяйственный отдел
служба ит
отдел архитектура
служба ит
отдел разработка система
отдел архитектура
бухгалтерия
административный - хозяйственный отдел
бухгалтерия
административный - хозяйственный отдел
бухгалтерия
бухгалтерия
отдел разработка система
служба ит
отдел разработка система
административный - хозяйственный отдел
отдел разработка система
административный - хозяйственный отдел
отдел эксплуатация
отдел проектирование
служба ит
отдел проектирование
бухгалтерия
отдел проектирование
отдел разработка система
отдел разработка система
отдел эксплуатация
отдел разработка система
отдел разработка система
административный - хозяйственный отдел
бухгалтерия
отдел разработка система
отдел разработка система
бухгалтерия
административный - хозяйственный отдел
служба ит
отдел проектиро

In [ ]:
len(train_groups)

8983

In [59]:
nlp.remove_pipe('ner') # удаляем NER 

('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f193de8a270>)

# Обучаем spacy находить новые NER

Создаем собственную NER == GROUP

In [61]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("ru")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [62]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

train = train_groups[:110]
nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

Losses at iteration 0 - 2023-01-19 03:55:10.333719 {'ner': 1797.3166107069183}
Losses at iteration 1 - 2023-01-19 03:55:20.521652 {'ner': 258.78387471201586}
Losses at iteration 2 - 2023-01-19 03:55:31.569032 {'ner': 270.02742560007823}
Losses at iteration 3 - 2023-01-19 03:55:42.108721 {'ner': 251.31472052753273}
Losses at iteration 4 - 2023-01-19 03:55:53.371046 {'ner': 247.3522450343645}
Losses at iteration 5 - 2023-01-19 03:56:03.045806 {'ner': 276.6066268905294}
Losses at iteration 6 - 2023-01-19 03:56:12.464182 {'ner': 232.06385972132952}
Losses at iteration 7 - 2023-01-19 03:56:22.019985 {'ner': 240.7034382930505}
Losses at iteration 8 - 2023-01-19 03:56:31.215232 {'ner': 234.9035906480857}


### Проверяем

In [63]:
def get_post(x):
  doc = nlp(x)
  for ent in doc.ents:
    print(ent)
    return ent.text

In [64]:
 str = """ооо ' мегасофт " приказ в связь с окончание проект бухгалтерия внедрение система знание  поручать обеспечить выполнение задача : 1 ) провести анализ неисправность работа api и обеспечить устранение ошибка . установить срок выполнение задача до 11.04.13 . контроль над выполнение приказ возложить на технический писатель абалкин зураб i. выполнить настройка канал для подключение к сеть интернет . выполнить данное поручение до 24 март 14 . ответственность за исполнение распоряжение возложить на специалист второй категория отдел разработка система попов а.а. 1 . провести анализ накопить технический долг и определить приоритет исправление замечание . выполнить данное поручение до 26 июль 2019 год . за контроль и выполнение назначить ответственный специалист первый категория отдел разработка система никишин с.е. 2 ) расширить функциональность api с учёт обновить технический задание . срок завершение : 04 мар 2017 г .. контроль над выполнение настоящий приказ возложить на тимлид проектный команда"""

In [65]:
print(get_post(str))

отдел разработка система
отдел разработка система


In [67]:
nlp.to_disk("/content/drive/MyDrive/Хакатон_2023/spacy/ner_group1")

# Добавляем исполнителей по задачам на основании новой сущности "POST" в spacy

In [68]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("ru")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [69]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

train = train_posts[:210]
nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

Losses at iteration 0 - 2023-01-19 04:01:30.772693 {'ner': 1775.7069966536367}
Losses at iteration 1 - 2023-01-19 04:01:44.869404 {'ner': 330.2995323209161}
Losses at iteration 2 - 2023-01-19 04:01:59.476598 {'ner': 306.14524905485496}
Losses at iteration 3 - 2023-01-19 04:02:12.887196 {'ner': 335.15179752750385}
Losses at iteration 4 - 2023-01-19 04:02:26.304409 {'ner': 323.2924458341528}
Losses at iteration 5 - 2023-01-19 04:02:39.104025 {'ner': 312.8149688776195}
Losses at iteration 6 - 2023-01-19 04:02:52.415045 {'ner': 306.3076752247097}
Losses at iteration 7 - 2023-01-19 04:03:05.603618 {'ner': 312.9538707215999}
Losses at iteration 8 - 2023-01-19 04:03:20.505477 {'ner': 307.5063040219062}


In [70]:
 str = """ооо ' мегасофт " приказ в связь с окончание проект бухгалтерия внедрение система знание  поручать обеспечить выполнение задача : 1 ) провести анализ неисправность работа api и обеспечить устранение ошибка . установить срок выполнение задача до 11.04.13 . контроль над выполнение приказ возложить на технический писатель абалкин зураб i. выполнить настройка канал для подключение к сеть интернет . выполнить данное поручение до 24 март 14 . ответственность за исполнение распоряжение возложить на специалист второй категория отдел разработка система попов а.а. 1 . провести анализ накопить технический долг и определить приоритет исправление замечание . выполнить данное поручение до 26 июль 2019 год . за контроль и выполнение назначить ответственный специалист первый категория отдел разработка система никишин с.е. 2 ) расширить функциональность api с учёт обновить технический задание . срок завершение : 04 мар 2017 г .. контроль над выполнение настоящий приказ возложить на тимлид проектный команда"""

In [71]:
print(get_post(str))

специалист первый категория
специалист первый категория


In [72]:
nlp.to_disk("/content/drive/MyDrive/Хакатон_2023/spacy/ner_post1")

In [ ]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("ru")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [ ]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

train = train_titles[:110]
nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

Losses at iteration 0 - 2023-01-18 20:11:20.310415 {'ner': 1229.0599063780764}
Losses at iteration 1 - 2023-01-18 20:11:26.247436 {'ner': 0.519154753585956}
Losses at iteration 2 - 2023-01-18 20:11:32.814149 {'ner': 0.4573916494315827}
Losses at iteration 3 - 2023-01-18 20:11:38.706434 {'ner': 2.7841031970065736e-06}
Losses at iteration 4 - 2023-01-18 20:11:44.604014 {'ner': 2.3983217552087464}
Losses at iteration 5 - 2023-01-18 20:11:50.622550 {'ner': 0.0005870606450456637}
Losses at iteration 6 - 2023-01-18 20:11:56.627675 {'ner': 1.6414663688496558e-05}
Losses at iteration 7 - 2023-01-18 20:12:02.662377 {'ner': 1.8595737789415788e-07}
Losses at iteration 8 - 2023-01-18 20:12:08.679109 {'ner': 2.7910095514335094e-06}


In [ ]:
 str = """ооо ' мегасофт " приказ в связь с окончание проект бухгалтерия внедрение система знание  поручать обеспечить выполнение задача : 1 ) провести анализ неисправность работа api и обеспечить устранение ошибка . установить срок выполнение задача до 11.04.13 . контроль над выполнение приказ возложить на технический писатель абалкин зураб i. выполнить настройка канал для подключение к сеть интернет . выполнить данное поручение до 24 март 14 . ответственность за исполнение распоряжение возложить на специалист второй категория отдел разработка система попов а.а. 1 . провести анализ накопить технический долг и определить приоритет исправление замечание . выполнить данное поручение до 26 июль 2019 год . за контроль и выполнение назначить ответственный специалист первый категория отдел разработка система никишин с.е. 2 ) расширить функциональность api с учёт обновить технический задание . срок завершение : 04 мар 2017 г .. контроль над выполнение настоящий приказ возложить на тимлид проектный команда"""

In [ ]:
print(get_post(str))

приказ
приказ


In [ ]:
nlp.to_disk("/content/drive/MyDrive/ДЗ/ner_title3")

In [ ]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("ru")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [ ]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

train = train_signatory[:550]
nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

Losses at iteration 0 - 2023-01-18 20:14:41.383644 {'ner': 980.9723495464975}
Losses at iteration 1 - 2023-01-18 20:15:11.578768 {'ner': 0.00020026045216568875}
Losses at iteration 2 - 2023-01-18 20:15:43.418491 {'ner': 1.5001168833629286}
Losses at iteration 3 - 2023-01-18 20:16:14.283835 {'ner': 6.973663063256632}
Losses at iteration 4 - 2023-01-18 20:16:44.738419 {'ner': 3.219760045181809e-05}
Losses at iteration 5 - 2023-01-18 20:17:15.481752 {'ner': 26.117643740379844}
Losses at iteration 6 - 2023-01-18 20:17:46.629050 {'ner': 0.0006299892396222957}
Losses at iteration 7 - 2023-01-18 20:18:21.427663 {'ner': 1.998933511983235e-05}
Losses at iteration 8 - 2023-01-18 20:18:52.577902 {'ner': 16.553132302106075}


In [ ]:
 str = """'ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : 1 . провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 . ответственность за исполнение распоряжение возложить на специалист третий категория служба ит ким юрий иванович 2 . выполнить ремонт кабель сеть передача данных между этаж здание . дата окончание : 12 авг 13 . контроль над выполнение приказ возложить на системный администратор сергеев андрей 3 . создать и настроить виртуальный машина в соответствие с технический задание . установить срок выполнение задача до 10 сен 19 . контроль над исполнение распоряжение оставить за специалист третий категория служба ит ким юрий иванович и.о. генеральный директор сердитый а.и. 03 июнь 2013 год'"""

In [ ]:
print(get_post(str))

и.о. генеральный директор сердитый а.и.
и.о. генеральный директор сердитый а.и.


In [ ]:
nlp.to_disk("/content/drive/MyDrive/ДЗ/ner_sig4")

In [73]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("ru")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [74]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

train = train_tasks
nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

Losses at iteration 0 - 2023-01-19 04:04:04.008111 {'ner': 2080.292683683729}
Losses at iteration 1 - 2023-01-19 04:04:09.983575 {'ner': 215.32569441622073}
Losses at iteration 2 - 2023-01-19 04:04:17.325584 {'ner': 68.37844196015786}
Losses at iteration 3 - 2023-01-19 04:04:23.248414 {'ner': 37.97188625138518}
Losses at iteration 4 - 2023-01-19 04:04:29.446848 {'ner': 8.560828710240271}
Losses at iteration 5 - 2023-01-19 04:04:35.510725 {'ner': 6.767601119118394}
Losses at iteration 6 - 2023-01-19 04:04:41.496614 {'ner': 3.549855566402118}
Losses at iteration 7 - 2023-01-19 04:04:47.522619 {'ner': 3.8106960917928983}
Losses at iteration 8 - 2023-01-19 04:04:54.764433 {'ner': 4.485200581918595}


In [75]:
 str = """'ооо " мегасофт " приказ для организация мероприятие " день заказчик " приказывать : 1 . провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 . ответственность за исполнение распоряжение возложить на специалист третий категория служба ит ким юрий иванович 2 . выполнить ремонт кабель сеть передача данных между этаж здание . дата окончание : 12 авг 13 . контроль над выполнение приказ возложить на системный администратор сергеев андрей 3 . создать и настроить виртуальный машина в соответствие с технический задание . установить срок выполнение задача до 10 сен 19 . контроль над исполнение распоряжение оставить за специалист третий категория служба ит ким юрий иванович и.о. генеральный директор сердитый а.и. 03 июнь 2013 год'"""

In [76]:
print(get_post(str))

провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 .
провести анализ поставщик для замена выйти из строй оборудование . срок завершение : 24 октябрь 12 .


In [77]:
nlp.to_disk("/content/drive/MyDrive/Хакатон_2023/spacy/ner_tasks5")